In [1]:

import sys

import matplotlib.pyplot as plt
import pandas as pd
import torch

from data_factory.DataLoader import DataLoader
from utils.config import load_config
# evaluation file
from utils.evaluation import label_evaluation

import numpy as np

sys.path.append('/media/cyprien/Data/Documents/Github/pytorch-forecasting')



In [2]:
config_file = "../config/config.yml"
config = load_config(config_file)

result_file = '../predict.csv'
truth_file = '../ground_truth.hdf'

In [3]:
dl = DataLoader(use_previous_files=True, config_file=config_file)

INFO:data_factory.DataLoader:Use previously generated file ../data//data_export_train.csv_test.csv_fmd-True_False_True_True_0.95.p. Can not redo preprocessing by loading from generated file.


## Feature Engineering

In [4]:
max_prediction_length = 1
max_encoder_length = config['AnomalyTransformer']['max_seq_len']
min_encoder_length = config['AnomalyTransformer']['min_seq_len']


In [5]:
from pytorch_forecasting import TimeSeriesDataSet

X_cols = ['value_scaled', 'kpi_id', 'timestamp_1', 'authentic']
normal_train = dl.train.reset_index(drop=True)

training = TimeSeriesDataSet(
    normal_train.loc[:, X_cols],
    time_idx='timestamp_1', target='value_scaled',
    group_ids=['kpi_id'],
    allow_missing_timesteps=False,
    static_categoricals=['kpi_id', 'authentic'],
    time_varying_unknown_reals=['value_scaled'],
    # time_varying_known_reals=['timestamp_1'],
    max_encoder_length=max_encoder_length,
    min_encoder_length=min_encoder_length,
    max_prediction_length=max_prediction_length,
    scalers={col: None for col in ['timestamp_1', 'kpi_id']},
    target_normalizer=None,
    add_relative_time_idx=False,
    add_target_scales=False,
    add_encoder_length=False,
)

validation = TimeSeriesDataSet.from_dataset(
    training, dl.val.loc[:, X_cols], stop_randomization=True, predict=False
)
testing = TimeSeriesDataSet.from_dataset(
    training, dl.test.loc[:, X_cols], stop_randomization=True, predict=False, min_encoder_length=max_encoder_length
)

In [6]:
batch_size = 64

training_dl = training.to_dataloader(train=True, batch_size=batch_size, num_workers=12)

validation_dl = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=12)
testing_dl = testing.to_dataloader(train=False, batch_size=batch_size * 3, num_workers=12)


## Model

In [7]:
from models.AnomalyTransformer.AnomalyTransformer import AnomalyTransformer

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using", device)
n_features = 1

Using cuda


In [9]:
try:
    del model
except NameError:
    pass

try:
    del trainer
except NameError:
    pass

torch.cuda.empty_cache()


In [10]:

from tqdm import tqdm

model = AnomalyTransformer(
    win_size=max_encoder_length, enc_in=n_features, c_out=1,
    d_model=256, n_heads=4, e_layers=2, d_ff=256,
    dropout=0.0, activation='gelu', output_attention=True
)

# model = LitAE.load_from_checkpoint("./lightning_logs/version_1/checkpoints/epoch=4-step=171145.ckpt",
#                                    input_shape=max_encoder_length, n_dim=n_dim)


INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpiqec14bp
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpiqec14bp/_remote_module_non_sriptable.py


In [11]:
from pytorch_lightning.trainer import Trainer

trainer = Trainer(logger=True, enable_checkpointing=True, checkpoint_callback=None, gpus=1, auto_lr_find=True, max_epochs=-1)

trainer.validate(model=model, dataloaders=validation_dl)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     val_loss_epoch          3.32279896736145
   val_max_loss_epoch        25.94574737548828
   val_min_loss_epoch       -22.622953414916992
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_loss_epoch': 3.32279896736145,
  'val_min_loss_epoch': -22.622953414916992,
  'val_max_loss_epoch': 25.94574737548828}]

In [ ]:
trainer.fit(model=model, train_dataloaders=training_dl, val_dataloaders=validation_dl)

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type          | Params
---------------------------------------------
0 | embedding  | DataEmbedding | 768   
1 | encoder    | Encoder       | 795 K 
2 | projection | Linear        | 257   
---------------------------------------------
796 K     Trainable params
0         Non-trainable params
796 K     Total params
3.185     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

In [ ]:
trainer.validate(model=model, dataloaders=validation_dl)

# trainer.test(model)
# trainer.predict(model)

## Predict abnormal value

In [ ]:
from tqdm.notebook import tqdm

def predict_value(df: pd.DataFrame):
    df['timestamp_1_floor'] = df.groupby('kpi_id').timestamp_1.transform(lambda x: x - x.min())
    pv = pd.pivot_table(df, values='value_scaled', index='timestamp_1_floor', columns='kpi_id', fill_value=np.nan)

    ept = np.empty(max_encoder_length)
    ept[:] = np.nan

    pv = pd.concat([pv, pd.DataFrame({col: ept.copy() for col in pv.columns})])

    pv_forward = np.moveaxis(pv.copy().to_numpy(dtype=np.float32), 1, 0)
    pv_filter = np.moveaxis(pv.copy().to_numpy(), 1, 0)
    pv_forward[np.isnan(pv_forward)] = 0
    pv_filter = np.where(np.isnan(pv_filter), 0, 1)

    df['value_pred'] = np.nan

    for i in tqdm(np.arange(0, pv.index.max() + 1, max_encoder_length)):
        pv_forward_i = pv_forward[:, i:i + max_encoder_length]
        pv_filter_i = pv_filter[:, i:i + max_encoder_length]

        X = torch.from_numpy(pv_forward_i.copy())
        X = X.reshape(X.shape[0], X.shape[1], 1)

        y_pred = model(X)

        validation_filter = df.groupby('kpi_id').apply(
            lambda x: (x.timestamp_1_floor >= i) & (x.timestamp_1_floor < i + max_encoder_length)).reset_index(
            drop=True)

        df.loc[validation_filter, 'value_pred'] = y_pred.cpu().detach().numpy()[pv_filter_i.astype(bool)].flatten()


predict_value(dl.train)
predict_value(dl.test)


In [ ]:
def predict_label(df: pd.DataFrame):

    df['label_pred'] = np.nan

    for kpi_id in tqdm(df.kpi_id.unique()):
        THRESHOLD = np.abs(df[(df.kpi_id == kpi_id)].value_pred - df[(df.kpi_id == kpi_id)].value_scaled).quantile(.991)
        df.loc[(df.kpi_id == kpi_id), 'label_pred'] = np.abs(
            df[(df.kpi_id == kpi_id)].value_pred -
            df[(df.kpi_id == kpi_id)].value_scaled) > THRESHOLD


predict_label(dl.train)
predict_label(dl.test)

### Training

In [ ]:
import logging
from logging import WARNING

logging.basicConfig(level=WARNING)

for kpi_id in dl.train.kpi_id.unique()[:3]:
    df = dl.train[(dl.train.kpi_id == kpi_id) & (~validation_filter)]

    figure = plt.figure(figsize=(20, 4))
    plt.hist(np.abs(df.value_pred - df.value_scaled), bins=200, figure=figure)

    true_positive = ((df.label == 1) & (df.label_pred == 1))
    false_positive = ((df.label == 0) & (df.label_pred == 1))
    false_negative = ((df.label == 1) & (df.label_pred == 0))

    figure = plt.figure(figsize=(20, 8))
    for filter, color in zip([true_positive, false_positive, false_negative], ['g', 'orange', 'r']):
        plt.scatter(df[filter].timestamp_1, df[filter].value_scaled, c=color, alpha=.4)

    plt.plot(df.timestamp_1, df.value_scaled, figure=figure, linewidth=.5)
    plt.plot(df.timestamp_1, df.value_pred, figure=figure, linewidth=.5)
    plt.title(f'kpi id: {kpi_id}')
    plt.show()


In [ ]:
import logging
from logging import WARNING

logging.basicConfig(level=WARNING)

for kpi_id in dl.train.kpi_id.unique()[:3]:
    df = dl.train[(dl.train.kpi_id == kpi_id) & (validation_filter)]

    figure = plt.figure(figsize=(20, 4))
    plt.hist(np.abs(df.value_pred - df.value_scaled), bins=200, figure=figure)

    true_positive = ((df.label == 1) & (df.label_pred == 1))
    false_positive = ((df.label == 0) & (df.label_pred == 1))
    false_negative = ((df.label == 1) & (df.label_pred == 0))

    figure = plt.figure(figsize=(20, 8))
    for filter, color in zip([true_positive, false_positive, false_negative], ['g', 'orange', 'r']):
        plt.scatter(df[filter].timestamp_1, df[filter].value_scaled, c=color, alpha=.4)

    plt.plot(df.timestamp_1, df.value_scaled, figure=figure, linewidth=.5)
    plt.plot(df.timestamp_1, df.value_pred, figure=figure, linewidth=.5)
    plt.title(f'kpi id: {kpi_id}')
    plt.show()


## Export

In [ ]:
validation_start_points = normal_train.groupby('kpi_id').apply(
    lambda x: x.timestamp_1.max() - (max_encoder_length + max_prediction_length)).to_frame('limit').to_dict()['limit']
validation_start_points

In [ ]:
validation_filter = dl.train.groupby('kpi_id').apply(
    lambda x: x.timestamp_1 >= validation_start_points[x.kpi_id.iloc[0]]).reset_index(
    drop=True)

In [ ]:
prediction = dl.train.loc[validation_filter  & (dl.train.authentic == 'True'), ['timestamp', 'kpi_id', 'label_pred']].rename(columns={'kpi_id': 'KPI ID'})
prediction['predict'] = prediction['label_pred']
prediction.to_csv(result_file)

ground_truth = dl.train.loc[validation_filter & (dl.train.authentic == 'True'), ['timestamp', 'kpi_id', 'label']].rename(columns={'kpi_id': 'KPI ID'})
ground_truth.to_hdf(truth_file, key='df')

print(label_evaluation(truth_file, result_file))

In [ ]:
prediction = dl.train.loc[~validation_filter & (dl.train.authentic == 'True'), ['timestamp', 'kpi_id', 'label_pred']].rename(columns={'kpi_id': 'KPI ID'})
prediction['predict'] = prediction['label_pred']
prediction.to_csv(result_file)

ground_truth = dl.train.loc[~validation_filter & (dl.train.authentic == 'True'), ['timestamp', 'kpi_id', 'label']].rename(columns={'kpi_id': 'KPI ID'})
ground_truth.to_hdf(truth_file, key='df')

print(label_evaluation(truth_file, result_file))

.997: .2791
.995: .4579
.993: .4628
.992: .5719
.991: .5609
.990: .5525
.989: .5521
.985: .5469

In [ ]:
prediction = dl.test.loc[dl.test.authentic == 'True', ['timestamp', 'kpi_id', 'label_pred']].rename(columns={'kpi_id': 'KPI ID'})
prediction['predict'] = prediction['label_pred'].astype(int)
prediction.drop(columns=['label_pred'], inplace=True)
prediction.to_csv(result_file)

### Plot

In [ ]:
import logging
from logging import WARNING

logging.basicConfig(level=WARNING)

for kpi_id in dl.test.kpi_id.unique()[:10]:
    df = dl.test[(dl.test.kpi_id == kpi_id) & (dl.test.authentic == 'True')]

    figure = plt.figure(figsize=(20, 4))
    plt.hist(np.abs(df.value_pred - df.value_scaled), bins=200, figure=figure)
    plt.show()

    figure = plt.figure(figsize=(20, 4))

    plt.scatter(df[df.label_pred == 1].timestamp_1, df[df.label_pred == 1].value_scaled, c='r', alpha=.4)

    plt.plot(df.timestamp_1, df.value_scaled, figure=figure, linewidth=.5)
    plt.plot(df.timestamp_1, df.value_pred, figure=figure, linewidth=.5)
    plt.title(f'kpi id: {kpi_id}')
    plt.show()